In [ ]:
from coscine import Client
from utils import CoscineOverview, DataExplorer, WorkCoscineOverview
from pyiron import Project

# (CoScInE) Metadata exploration

This notebook shows how to explore the metadata stored in some project on CoScInE.

Here you may specify your token to also access file contents on coscine (or to update the metadata with all resources currently accessible by you). The CoscineOverview is the basic interface to CoScInE and the WorkCoscineOverview gives access to metadata for specific Resources/Files/Schemes:

In [ ]:
co = CoscineOverview()

In [ ]:
co.download_from_coscine()

In [ ]:
meta_data_worker = WorkCoscineOverview(co)

In [ ]:
meta_data_worker.browser

In [ ]:
meta_data_worker.browser.get_group('Coscine Demo Project 2022-06')

## Working with the metadata

The with the `get_metadata` method, you get a dataframe collecting all the information from the selected schemes. In this case, all metadata from entries belonging to the 'Sample' schema.

In [ ]:
sample_df = meta_data_worker.get_metadata('Sample')

In [ ]:
sample_df

The DataExplorer takes one dataframe and provides a simple Excel like view of the data. In addition, it adds a plotting functionality (currently only ternary plots):

In [ ]:
de = DataExplorer(sample_df)
de

In [ ]:
h=sample_df[sample_df["ID"]=="S_10"]
h

# Analyse data

In [ ]:
nano_ind_df = meta_data_worker.get_metadata('NanoIndentation')

In [ ]:
nano_ind_df[nano_ind_df['Sample ID'] == 'S_10']

In [ ]:
nano_ind_data_file = meta_data_worker.get_file_handle(nano_ind_df[nano_ind_df['Sample ID'] == 'S_10'])
nano_ind_data_file

In [ ]:
pr = Project('test_data_job')
job = pr.create.job.ExperimentalDataJob('data')

In [ ]:
job.set_instrument(instrument_type='Indenter', E0 = 463.7, T0 = 373, C = 0.0441, poisson = .24,
                    daf = [1836675.0332, -31356.61652, 200.891289, -0.288717, 0.00016],
                    T = 273 + 50 )

In [ ]:
job.set_data(nano_ind_data_file, experiment_type='Nanoindentation', poisson=0.2)

In [ ]:
job.set_analysis('OliverPharr')

In [ ]:
job.run()

In [ ]:
ax = job.analysis.plot_hardness(output=job.output)

In [ ]:
job.input

In [ ]:
form  = nano_ind_data_file.metadata

In [ ]:
form['ID'] = form['ID'] + '_analysis'
form['Comments'] = 'Additional analysis using OliverPharr'

In [ ]:
form

In [ ]:
job.output.to_csv('S_10.csv')

In [ ]:
with open('S_10.csv', 'br') as f:
    nano_ind_data_file.resource.upload(f, form)

In [ ]:
nano_ind_data_file.resource['S_10.csv']